### Bert finetuning on MRPC data
Pytorch + TensorFlow 2.0

https://github.com/huggingface/transformers#quick-tour-tf-20-training-and-pytorch-interoperability

Paraphrasing task

In [1]:
import tensorflow as tf
import tensorflow_datasets
from transformers import *
%load_ext tensorboard
import datetime

In [2]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.3.0-dev20200601
[]


In [3]:
batch_size_train = 32
batch_size_val = 64
# Load dataset, tokenizer, model from pretrained model/vocabulary
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')
data = tensorflow_datasets.load('glue/mrpc')

INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: glue/mrpc/1.0.0
INFO:absl:Load dataset info from /tmp/tmp75x65nastfds
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.location from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset glue (/home/shivangi/tensorflow_datasets/glue/mrpc/1.0.0)


INFO:absl:Downloading https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc into /home/shivangi/tensorflow_datasets/downloads/fire.goog.com_v0_b_mtl-sent-repr.apps.com_o_2FjSIMlCiqs1QSmIykr4IRPnEHjPuGwAz5i40v8K9U0Z8.tsvalt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc.tmp.e7ca5ea2e0a3431584132688b79e306f...
INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt into /home/shivangi/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_trainfGxPZuQWGBti4Tbd1YNOwQr-OqxPejJ7gcp0Al6mlSk.txt.tmp.dc1d5b71f69243aeb1628e99b704a6a3...
INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt into /home/shivangi/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_test0PdekMcyqYR-w4Rx_d7OTryq0J3RlYRn4rAMajy9Mak.txt.tmp.6638762e938e4edbb1c642348b11dfcc...
/home/shivangi/ai_powe

Shuffling and writing examples to /home/shivangi/tensorflow_datasets/glue/mrpc/1.0.0.incompleteFOE6YP/glue-train.tfrecord


INFO:absl:Done writing /home/shivangi/tensorflow_datasets/glue/mrpc/1.0.0.incompleteFOE6YP/glue-train.tfrecord. Shard lengths: [3668]
INFO:absl:Generating split validation


Shuffling and writing examples to /home/shivangi/tensorflow_datasets/glue/mrpc/1.0.0.incompleteFOE6YP/glue-validation.tfrecord


INFO:absl:Done writing /home/shivangi/tensorflow_datasets/glue/mrpc/1.0.0.incompleteFOE6YP/glue-validation.tfrecord. Shard lengths: [408]
INFO:absl:Generating split test


Shuffling and writing examples to /home/shivangi/tensorflow_datasets/glue/mrpc/1.0.0.incompleteFOE6YP/glue-test.tfrecord


INFO:absl:Done writing /home/shivangi/tensorflow_datasets/glue/mrpc/1.0.0.incompleteFOE6YP/glue-test.tfrecord. Shard lengths: [1725]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.AUTO.
INFO:absl:Constructing tf.data.Dataset for split None, from /home/shivangi/tensorflow_datasets/glue/mrpc/1.0.0


Dataset glue downloaded and prepared to /home/shivangi/tensorflow_datasets/glue/mrpc/1.0.0. Subsequent calls will reuse this data.


In [4]:
# Prepare dataset for GLUE as a tf.data.Dataset instance
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='mrpc')
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task='mrpc')
train_dataset = train_dataset.shuffle(100).batch(batch_size_train).repeat(2)
valid_dataset = valid_dataset.batch(batch_size_val)

In [5]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule

learning_rate = 3e-5 #1e-5 2e-5 3e-5 5e-5
epsilon = 1e-08

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [6]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [7]:
# Train and evaluate using tf.keras.Model.fit()

# history = model.fit(train_dataset, epochs=3, steps_per_epoch=115,
#                     validation_data=valid_dataset, validation_steps=7)
epochs = 1 #3
train_data_size = 3668
steps_per_epoch = int(train_data_size / batch_size_train)
history = model.fit(train_dataset, epochs=epochs, steps_per_epoch=steps_per_epoch,
                    validation_data=valid_dataset, validation_steps=7,
                   callbacks=[tensorboard_callback])

114/114 [==============================] - 493s 4s/step - loss: 0.5682 - accuracy: 0.6965 - val_loss: 0.4753 - val_accuracy: 0.7843


In [8]:
%tensorboard --logdir logs/fit

### end of training

In [26]:
# Load the TensorFlow model in PyTorch for inspection
model.save_pretrained('./save/')
pytorch_model = BertForSequenceClassification.from_pretrained('./save/', from_tf=True)

In [35]:
# Quickly test a few predictions - MRPC is a paraphrasing task, let's see if our model learned the task
sentence_0 = "This is wrong."
sentence_1 = "that was right."
sentence_2 = "This is incorrect."
inputs_1 = tokenizer.encode_plus(sentence_0, sentence_1, add_special_tokens=True, return_tensors='pt')
inputs_2 = tokenizer.encode_plus(sentence_0, sentence_2, add_special_tokens=True, return_tensors='pt')

pred_1 = pytorch_model(inputs_1['input_ids'], token_type_ids=inputs_1['token_type_ids'])[0].argmax().item()
pred_2 = pytorch_model(inputs_2['input_ids'], token_type_ids=inputs_2['token_type_ids'])[0].argmax().item()

print("sentence_1 is", "a paraphrase" if pred_1 else "not a paraphrase", "of sentence_0")
print("sentence_2 is", "a paraphrase" if pred_2 else "not a paraphrase", "of sentence_0")

sentence_1 is not a paraphrase of sentence_0
sentence_2 is a paraphrase of sentence_0


## trial and errors

In [7]:
import torch
from thop import profile

In [10]:
inputs = torch.randn(1, 1)
inputs = torch.tensor(inputs).to(torch.int64)
macs, params = profile(pytorch_model,(inputs,))

[WARN] Cannot find rule for <class 'torch.nn.modules.sparse.Embedding'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.normalization.LayerNorm'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[WARN] Cannot find rule for <class 'transformers.modeling_bert.BertEmbeddings'>. Treat it as zero Macs and zero Params.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'transformers.modeling_bert.BertSelfAttention'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'transformers.modeling_bert.BertSelfOutput'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'transformers.modeling_bert.BertAttention'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'transformers.modeling_bert.BertIntermediate'>. Treat it as zero Macs and zero Params.
[WARN] Cannot

/Users/shivangi/miniconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [11]:
print(macs, params)

171052032.0 85609730.0


In [12]:
model_new = BertModel.from_pretrained('bert-base-cased')

In [13]:
inputs = torch.randn(1, 1)
inputs = torch.tensor(inputs).to(torch.int64)
macs, params = profile(model_new,(inputs,))

[WARN] Cannot find rule for <class 'torch.nn.modules.sparse.Embedding'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.normalization.LayerNorm'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[WARN] Cannot find rule for <class 'transformers.modeling_bert.BertEmbeddings'>. Treat it as zero Macs and zero Params.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'transformers.modeling_bert.BertSelfAttention'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'transformers.modeling_bert.BertSelfOutput'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'transformers.modeling_bert.BertAttention'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'transformers.modeling_bert.BertIntermediate'>. Treat it as zero Macs and zero Params.
[WARN] Cannot

/Users/shivangi/miniconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [14]:
print(macs,params)

85524480.0 85608192.0


In [91]:
l = [module for module in model_new.modules() if type(module) != torch.nn.Sequential]

In [126]:
# l

In [122]:
params = list(model_new.named_parameters())

In [123]:
total = 0
for i in range(len(params)):
    #print (params[i][0], len(params[i][1]))
    total += len(params[i][1])
print (total)

235334
